In [0]:
%run ./Utils

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Create Database

In [0]:
%sql
CREATE DATABASE gold

## Import Data from silver layer

#### Step 1: trip type

In [0]:
df_trip_type = spark.read.format('parquet')\
                         .option('inferSchema', True)\
                         .option('header', True)\
                         .load(f'{silver_folder_path}/trip_type')

df_trip_type.display()

trip_type,trip_description
1,Street-hail
2,Dispatch


#### Step 2: taxi zone lookup

In [0]:
df_taxi_zone_lookup = spark.read.format('parquet')\
                         .option('inferSchema', True)\
                         .option('header', True)\
                         .load(f'{silver_folder_path}/taxi_zone_lookup')

df_taxi_zone_lookup.display()

LocationID,Borough,Zone,service_zone,zone_1,zone_2
1,EWR,Newark Airport,EWR,Newark Airport,null
2,Queens,Jamaica Bay,Boro,Jamaica Bay,null
3,Bronx,Allerton/Pelham Gardens,Boro,Allerton,Pelham Gardens
4,Manhattan,Alphabet City,Yellow,Alphabet City,null
5,Staten Island,Arden Heights,Boro,Arden Heights,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro,Arrochar,Fort Wadsworth
7,Queens,Astoria,Boro,Astoria,null
8,Queens,Astoria Park,Boro,Astoria Park,null
9,Queens,Auburndale,Boro,Auburndale,null
10,Queens,Baisley Park,Boro,Baisley Park,null


#### Step 3: trip data

In [0]:
df_trip = spark.read.format('parquet')\
                         .option('inferSchema', True)\
                         .option('header', True)\
                         .load(f'{silver_folder_path}/tripdata')

df_trip.display()

VendorID,PULocationID,DOLocationID,fare_amount,total_amount
2,65,49,9.3,13.8
2,7,179,7.2,11.64
2,74,42,6.5,9.0
2,75,235,25.4,32.9
2,256,49,12.1,17.52
1,210,210,9.3,12.8
2,66,4,19.8,28.05
2,95,95,13.5,16.0
2,24,143,12.8,21.05
2,210,210,8.0,9.0


## Create delta tables and create external tables on top of it

#### Step 1: trip type

In [0]:
df_trip_type.write.format('delta')\
    .mode('overwrite')\
    .option('path',f'{gold_folder_path}/trip_type')\
    .saveAsTable('gold.trip_type')

In [0]:
%sql
SELECT * FROM gold.trip_type;

trip_type,trip_description
1,Street-hail
2,Dispatch


#### Step 2: taxi zone lookup

In [0]:
df_taxi_zone_lookup.write.format('delta')\
    .mode('overwrite')\
    .option('path',f'{gold_folder_path}/taxi_zone_lookup')\
    .saveAsTable('gold.taxi_zone_lookup')

In [0]:
%sql
SELECT * FROM gold.taxi_zone_lookup;

LocationID,Borough,Zone,service_zone,zone_1,zone_2
1,EWR,Newark Airport,EWR,Newark Airport,null
2,Queens,Jamaica Bay,Boro,Jamaica Bay,null
3,Bronx,Allerton/Pelham Gardens,Boro,Allerton,Pelham Gardens
4,Manhattan,Alphabet City,Yellow,Alphabet City,null
5,Staten Island,Arden Heights,Boro,Arden Heights,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro,Arrochar,Fort Wadsworth
7,Queens,Astoria,Boro,Astoria,null
8,Queens,Astoria Park,Boro,Astoria Park,null
9,Queens,Auburndale,Boro,Auburndale,null
10,Queens,Baisley Park,Boro,Baisley Park,null


#### Step 3: trip data

In [0]:
df_trip.write.format('delta')\
    .mode('overwrite')\
    .option('path',f'{gold_folder_path}/tripdata')\
    .saveAsTable('gold.tripdata')

In [0]:
%sql
SELECT * FROM gold.tripdata;

VendorID,PULocationID,DOLocationID,fare_amount,total_amount
2,65,49,9.3,13.8
2,7,179,7.2,11.64
2,74,42,6.5,9.0
2,75,235,25.4,32.9
2,256,49,12.1,17.52
1,210,210,9.3,12.8
2,66,4,19.8,28.05
2,95,95,13.5,16.0
2,24,143,12.8,21.05
2,210,210,8.0,9.0


## Using the powerful functionalities of Delta Tables

#### Step 1: Update Operation on taxi_lookup_zone

In [0]:
%sql
UPDATE gold.taxi_zone_lookup
SET service_zone = concat(service_zone,' zone')

num_affected_rows
265


In [0]:
%sql
select * from gold.taxi_zone_lookup;

LocationID,Borough,Zone,service_zone,zone_1,zone_2
1,EWR,Newark Airport,EWR zone,Newark Airport,null
2,Queens,Jamaica Bay,Boro zone,Jamaica Bay,null
3,Bronx,Allerton/Pelham Gardens,Boro zone,Allerton,Pelham Gardens
4,Manhattan,Alphabet City,Yellow zone,Alphabet City,null
5,Staten Island,Arden Heights,Boro zone,Arden Heights,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro zone,Arrochar,Fort Wadsworth
7,Queens,Astoria,Boro zone,Astoria,null
8,Queens,Astoria Park,Boro zone,Astoria Park,null
9,Queens,Auburndale,Boro zone,Auburndale,null
10,Queens,Baisley Park,Boro zone,Baisley Park,null


#### Step 2: Check all the versions of the taxi_zone_lookup delta table

In [0]:
%sql
DESCRIBE HISTORY gold.taxi_zone_lookup;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-03-13T19:19:39.000Z,3787855952999020,sharihar1@hawk.iit.edu,UPDATE,Map(predicate -> []),null,null,0313-191721-8p29n5n4-v2n,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 9788, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3434, numDeletionVectorsUpdated -> 0, scanTimeMs -> 39, numAddedFiles -> 1, numUpdatedRows -> 265, numAddedBytes -> 9829, rewriteTimeMs -> 3379)",null,Databricks-Runtime/16.2.x-photon-scala2.12
0,2025-03-13T01:01:22.000Z,3787855952999020,sharihar1@hawk.iit.edu,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,null,0313-004923-uxck8gnv-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 265, numOutputBytes -> 9788)",null,Databricks-Runtime/16.2.x-photon-scala2.12


#### Step 3: Restore the table taxi_zone_lookup to version 0

In [0]:
%sql
RESTORE TABLE gold.taxi_zone_lookup TO VERSION AS OF 0;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
9788,1,1,1,9829,9788


In [0]:
%sql
SELECT * FROM gold.taxi_zone_lookup;

LocationID,Borough,Zone,service_zone,zone_1,zone_2
1,EWR,Newark Airport,EWR,Newark Airport,null
2,Queens,Jamaica Bay,Boro,Jamaica Bay,null
3,Bronx,Allerton/Pelham Gardens,Boro,Allerton,Pelham Gardens
4,Manhattan,Alphabet City,Yellow,Alphabet City,null
5,Staten Island,Arden Heights,Boro,Arden Heights,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro,Arrochar,Fort Wadsworth
7,Queens,Astoria,Boro,Astoria,null
8,Queens,Astoria Park,Boro,Astoria Park,null
9,Queens,Auburndale,Boro,Auburndale,null
10,Queens,Baisley Park,Boro,Baisley Park,null
